<a href="https://colab.research.google.com/github/NiklasElsaesser/FaceRecognition/blob/main/FaceBias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

\For labeling, the pictures have been structured in a folder structure which looks like this:

<img src="drawing.jpg" alt="drawing" width="200"/>

# Project "Bias in Face-Detection"


Installing necessary libraries
1.   OpenCV to preprocess the data (pictures)  
2.   numpy to organize the data and link it with the labels



In [1]:
!pip install opencv-contrib-python
!pip install numpy as np
!pip install tensorflow
!pip install wandb

ERROR: Could not find a version that satisfies the requirement as (from versions: none)
ERROR: No matching distribution found for as
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 30.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.7 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=dbcda063fd40a5fb83f4f2e35624b9ac8f520bd508bde551b86fa97a51da2dae
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


Import the W&B library and initialize your W&B project at the beginning of your Colab notebook:

In [2]:
import wandb
wandb.init(project="4facesbias")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Mounting a Google Drive where the pictures are uploaded from.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Now the pictures get loaded and preprocessed.

In [4]:
import os
import cv2
import numpy as np

data = []
labels = []

genders = ["men", "women"]
emotions = ["happy", "neutral"]

for gender in genders:
    for emotion in emotions:
        folder_path = f'/content/drive/MyDrive/Faces/Dataset/{gender}/{emotion}/'
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (48, 48))
            data.append(img)
            # Create labels by combining gender and emotion indices
            label = genders.index(gender) * len(emotions) + emotions.index(emotion)
            labels.append(label)

data = np.array(data)
data = data.reshape((data.shape[0], 48, 48, 1))
labels = np.array(labels)


Now Splitting the Data into a training and test set in an 80 - 20 config.

In [5]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

Building and training of the model.

In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.callbacks import ModelCheckpoint

model = Sequential()
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(48, 48, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(len(genders) * len(emotions), activation='softmax'))  # Output layer with appropriate number of units

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

wandb_callback = wandb.keras.WandbCallback()

model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=10, batch_size=64)

wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/10
3/3 [==============================] - 11s 241ms/step - loss: 165.3850 - accuracy: 0.4857 - val_loss: 29.6233 - val_accuracy: 0.5227
Epoch 2/10
3/3 [==============================] - 0s 20ms/step - loss: 28.6201 - accuracy: 0.4571 - val_loss: 3.5330 - val_accuracy: 0.4773
Epoch 3/10
3/3 [==============================] - 0s 20ms/step - loss: 2.0091 - accuracy: 0.6629 - val_loss: 0.6659 - val_accuracy: 0.7500
Epoch 4/10
3/3 [==============================] - 0s 20ms/step - loss: 0.5966 - accuracy: 0.7771 - val_loss: 0.1303 - val_accuracy: 0.9318
Epoch 5/10
3/3 [==============================] - 0s 20ms/step - loss: 0.1166 - accuracy: 0.9600 - val_loss: 0.0249 - val_accuracy: 1.0000
Epoch 6/10
3/3 [==============================] - 0s 20ms/step - loss: 0.0261 - accuracy: 1.0000 - val_loss: 0.0018 - val_accuracy: 1.0000
Epoch 7/10
3/3 [==============================] - 0s 20ms/step - loss: 0.0045 - accuracy: 1.0000 - val_loss: 2.7904e-04 - val_accuracy: 1.0000
Epoch 8/10
3/3 [=

In [8]:
wandb_callback = wandb.keras.WandbCallback()
#history = model.fit(train_images, train_labels, epochs=epochs, batch_size=batch_size, validation_data=(val_images, val_labels), callbacks=[wandb_callback])


NameError: ignored

In [9]:
test_images, test_labels = load_images('path_to_test_images')
predictions = model.predict(test_images)
# Process predictions and print results

NameError: ignored

checking for loss and accuracy.

In [10]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test Loss: {loss:.2f}")
print(f"Test Accuracy: {accuracy * 100:.2f}%")

2/2 [==============================] - 0s 89ms/step - loss: 7.2168e-06 - accuracy: 1.0000
Test Loss: 0.00
Test Accuracy: 100.00%


Saving the trained model to the GDrive

In [11]:
from google.colab import drive
drive.mount('/content/drive')

# Save the model to Google Drive
model.save('bias_model.keras')
#model.save('/content/drive/MyDrive/Faces/Dataset/emotion_model.h5')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


allow for the input of a new picture on which the algorithm wasnt trained to test for the bias of the algorithm.

In [12]:
from google.colab import files
import numpy as np
import cv2
from tensorflow.keras.models import load_model

genders = ["Male", "Female"]  # Example values, adjust based on your use case
emotions = ["Happy", "Sad", "Angry", "Surprised"]  # Example values, adjust based on your use case


# Load the trained model
model_path = '/content/drive/MyDrive/Faces/Dataset/emotion_model.h5'
model = load_model(model_path)

# Upload your own picture
uploaded = '/content/drive/MyDrive/Faces/testing'

# Convert the uploaded files to a dictionary
#uploaded_files = dict(uploaded)

# Get the file paths of the uploaded images
#test_image_paths = list(uploaded_files.keys())


# Function to preprocess uploaded images
def preprocess_image(file_path):
    img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if img is None:
        raise ValueError(f"Image not loaded successfully from {file_path}")
    img = cv2.resize(img, (48, 48))
    if img.shape != (48, 48):
        raise ValueError(f"Image size is not 48x48 after resizing")

    img = np.expand_dims(img, axis=0)
    img = img.reshape((1, 48, 48, 1))
    return img

# Function to predict emotion based on the uploaded image
def predict_emotion(file_path):
    preprocessed_img = preprocess_image(file_path)
    prediction = model.predict(preprocessed_img)
    gender_index, emotion_index = np.unravel_index(np.argmax(prediction, axis=None), prediction.shape)
    gender = genders[gender_index]
    emotion = emotions[emotion_index % len(emotions)]
    return gender, emotion



# Process and predict emotions for the uploaded image(s)
for file_name in uploaded.keys():
    file_path = '/content/' + file_name #add correct file path
    gender, emotion = predict_emotion(file_path)
    print(f"Predicted Gender: {gender.capitalize()}, Predicted Emotion: {emotion.capitalize()}")


AttributeError: ignored